In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
NUM_EXPERIENCES = 10
# DATA_PATH = f"./logs/ST_SplitCIFAR100_{NUM_EXPERIENCES}"
# DATA_PATH = f"./logs/F_SplitCIFAR100_2"
# DATA_PATH = f"./logs/SplitCIFAR100_lwf"
# DATA_PATH = f"./logs/SplitCIFAR100_DER_500"
# DATA_PATH = f"./logs/SplitCIFAR100_DER_2000"
# DATA_PATH = f"./logs/SplitCIFAR100_DER_500_2"
# DATA_PATH = f"./logs/SplitCIFAR100_DER_2000_2"
# DATA_PATH = f"./logs/SplitCIFAR100_DER_RESNET"
# DATA_PATH = f"./logs/SplitCIFAR100_D_ST"
# DATA_PATH = f"./logs/SplitCIFAR100_D_HP"
# DATA_PATH = f"./logs/SplitCIFAR100_CHECK"

# DATA_PATH = f"./logs/SplitCIFAR100_J"
DATA_PATH = f"./logs/SplitCIFAR100_D"
# DATA_PATH = f"./logs/SplitCIFAR100_R"
# DATA_PATH = f"./logs/SplitCIFAR100_N"

DATASET = "SplitCIFAR100"
# MODEL_NAME = "ResNet18"
MODEL_NAME = "ResNet110"
valid_colors = ['green', 'red', 'cyan', 'magenta', 'black', 'purple', 'orange', 'brown', 'gray', 'olive', 'indigo', 'turquoise', 'green', 'red', 'cyan', 'magenta']*2

In [ ]:
# all combinations of training
run2name = {    
    # "j_nst_npp" : "JointTraining_NoSelfTraining_NoPostProcessing",
    # # "j_st_npp_1" : "JointTraining_SelfTraining1.0_NoPostProcessing",
    # "j_st_npp_01" : "JointTraining_SelfTraining_0.1_NoPostProcessing", #
    # # "j_st_npp_0075" : "JointTraining_SelfTraining0.075_NoPostProcessing",
    # "j_st_npp_005" : "JointTraining_SelfTraining_0.05_NoPostProcessing", #
    # "j_st_npp_0025" : "JointTraining_SelfTraining_0.025_NoPostProcessing", #
    # # "j_st_npp_001" : "JointTraining_SelfTraining0.01_NoPostProcessing",
    # # "j_st_npp_00075" : "JointTraining_SelfTraining0.0075_NoPostProcessing",
    # # "j_st_npp_0005" : "JointTraining_SelfTraining0.005_NoPostProcessing",
    # # "j_st_npp_00025" : "JointTraining_SelfTraining0.0025_NoPostProcessing",
    # # "j_st_npp" : "JointTraining_SelfTraining_NoPostProcessing", # 0.001
    # # "j_nst_pp" : "JointTraining_NoSelfTraining_PostProcessing"

    # "j_nst_npp_v1" : "JointTraining_NoSelfTraining_NoPostProcessing1",
    # "j_nst_npp_v2" : "JointTraining_NoSelfTraining_NoPostProcessing2",
    # "j_nst_npp_v3" : "JointTraining_NoSelfTraining_NoPostProcessing3",
    # "j_st_npp_0025_v1" : "JointTraining_SelfTraining_0.025_NoPostProcessing1",
    # "j_st_npp_0025_v2" : "JointTraining_SelfTraining_0.025_NoPostProcessing2",
    # "j_st_npp_0025_v3" : "JointTraining_SelfTraining_0.025_NoPostProcessing3",

    # "j_nst_npp_v1" : "JointTraining_NoSelfTraining_NoPostProcessing1",
    # "j_nst_npp_v2" : "JointTraining_NoSelfTraining_NoPostProcessing2",
    # "j_nst_npp_v3" : "JointTraining_NoSelfTraining_NoPostProcessing3",

    # "r_nst_npp_v1" : "Replay_NoSelfTraining_NoPostProcessing1",
    # "r_nst_npp_v2" : "Replay_NoSelfTraining_NoPostProcessing2",
    # "r_nst_npp_v3" : "Replay_NoSelfTraining_NoPostProcessing3",

    # "n_nst_npp_v1" : "Naive_NoSelfTraining_NoPostProcessing1",
    # "n_nst_npp_v2" : "Naive_NoSelfTraining_NoPostProcessing2",
    # "n_nst_npp_v3" : "Naive_NoSelfTraining_NoPostProcessing3",

    # "d_nst_npp_hp1" : "DER_NoSelfTraining_NoPostProcessing1",
    # "d_nst_npp_hp2" : "DER_NoSelfTraining_NoPostProcessing2",
    # "d_nst_npp_hp3" : "DER_NoSelfTraining_NoPostProcessing3",
    # "d_nst_npp_hp4" : "DER_NoSelfTraining_NoPostProcessing4",
    # "d_nst_npp_hp5" : "DER_NoSelfTraining_NoPostProcessing5",
    # "d_nst_npp_hp6" : "DER_NoSelfTraining_NoPostProcessing6",
    # "d_nst_npp_hp7" : "DER_NoSelfTraining_NoPostProcessing7",
    # "d_nst_npp_hp8" : "DER_NoSelfTraining_NoPostProcessing8",
    # "d_nst_npp_hp9" : "DER_NoSelfTraining_NoPostProcessing9",
    # "d_nst_npp_hp10" : "DER_NoSelfTraining_NoPostProcessing10",
    # "d_nst_npp_hp11" : "DER_NoSelfTraining_NoPostProcessing11",
    # "d_nst_npp_hp12" : "DER_NoSelfTraining_NoPostProcessing12",
    # "d_nst_npp_hp13" : "DER_NoSelfTraining_NoPostProcessing13",
    # "d_nst_npp_hp14" : "DER_NoSelfTraining_NoPostProcessing14",

    # "d_nst_npp_m1" : "DER_NoSelfTraining_NoPostProcessing1",
    # "d_nst_npp_m2" : "DER_NoSelfTraining_NoPostProcessing2",
    # "d_nst_npp_m3" : "DER_NoSelfTraining_NoPostProcessing3",
    # "d_nst_npp_m4" : "DER_NoSelfTraining_NoPostProcessing4",
    # "d_nst_npp_m5" : "DER_NoSelfTraining_NoPostProcessing5",
    # "d_nst_npp_m6" : "DER_NoSelfTraining_NoPostProcessing6",
    # "d_nst_npp_m7" : "DER_NoSelfTraining_NoPostProcessing7",
    # "d_nst_npp_m8" : "DER_NoSelfTraining_NoPostProcessing8",
    # "d_nst_npp_m9" : "DER_NoSelfTraining_NoPostProcessing9",
    # "d_nst_npp_m10" : "DER_NoSelfTraining_NoPostProcessing10",

    # "d_nst_npp_v1" : "DER_NoSelfTraining_NoPostProcessing1",
    # "d_nst_npp_v2" : "DER_NoSelfTraining_NoPostProcessing2",
    # "d_nst_npp_v3" : "DER_NoSelfTraining_NoPostProcessing3",

    # "d_st_npp_01" : "DER_SelfTraining_0.1_NoPostProcessing1",
    # "d_st_npp_0075" : "DER_SelfTraining_0.075_NoPostProcessing1",
    # "d_st_npp_005" : "DER_SelfTraining_0.05_NoPostProcessing1",
    # "d_st_npp_0025_v1" : "DER_SelfTraining_0.025_NoPostProcessing1",
    # "d_st_npp_001" : "DER_SelfTraining_0.01_NoPostProcessing1",
    # "d_st_npp_00075" : "DER_SelfTraining_0.0075_NoPostProcessing1",
    # "d_st_npp_0005" : "DER_SelfTraining_0.005_NoPostProcessing1",
    # "d_st_npp_00025" : "DER_SelfTraining_0.0025_NoPostProcessing1",

    # "d_nst_npp_v1" : "DER_NoSelfTraining_NoPostProcessing1",
    # "d_nst_npp_v2" : "DER_NoSelfTraining_NoPostProcessing2",
    # "d_nst_npp_v3" : "DER_NoSelfTraining_NoPostProcessing3",
    # "d_nst_npp_v4" : "DER_NoSelfTraining_NoPostProcessing4",
    # "d_nst_npp_v5" : "DER_NoSelfTraining_NoPostProcessing5",
    # "d_nst_npp_v6" : "DER_NoSelfTraining_NoPostProcessing6",
    # "d_nst_npp_v7" : "DER_NoSelfTraining_NoPostProcessing7",
    # "d_nst_npp_v8" : "DER_NoSelfTraining_NoPostProcessing8",
    # "d_nst_npp_v9" : "DER_NoSelfTraining_NoPostProcessing9",
    # "d_nst_npp_v10" : "DER_NoSelfTraining_NoPostProcessing10",
    # "d_nst_npp_v11" : "DER_NoSelfTraining_NoPostProcessing11",
    # "d_nst_npp_v12" : "DER_NoSelfTraining_NoPostProcessing12",
    # "d_nst_npp_v13" : "DER_NoSelfTraining_NoPostProcessing13",
    # "d_nst_npp_v14" : "DER_NoSelfTraining_NoPostProcessing14",
    # "d_nst_npp_v15" : "DER_NoSelfTraining_NoPostProcessing15",
    # "d_nst_npp_v16" : "DER_NoSelfTraining_NoPostProcessing16",
    # "d_nst_npp_v17" : "DER_NoSelfTraining_NoPostProcessing17",
    # "d_nst_npp_v18" : "DER_NoSelfTraining_NoPostProcessing18",
    # "d_nst_npp_v19" : "DER_NoSelfTraining_NoPostProcessing19",
    # "d_nst_npp_v20" : "DER_NoSelfTraining_NoPostProcessing20",
    # "d_nst_npp_v21" : "DER_NoSelfTraining_NoPostProcessing21",
    # "d_nst_npp_v22" : "DER_NoSelfTraining_NoPostProcessing22",
    # "d_nst_npp_v23" : "DER_NoSelfTraining_NoPostProcessing23",
    # "d_nst_npp_v24" : "DER_NoSelfTraining_NoPostProcessing24",

    # "j_nst_npp_v1" : "JointTraining_NoSelfTraining_NoPostProcessing1",
    # "j_nst_npp_v2" : "JointTraining_NoSelfTraining_NoPostProcessing0",
    # "j_st_npp_0025_v1" : "JointTraining_SelfTraining_0.025_NoPostProcessing1",
    # "j_st_npp_0025_v2" : "JointTraining_SelfTraining_0.025_NoPostProcessing0",
    # "j_nst_pp_ts_v1" : "JointTraining_NoSelfTraining_PostProcessing_TemperatureScaling1",
    # "j_nst_pp_ts_v2" : "JointTraining_NoSelfTraining_PostProcessing_TemperatureScaling0",
    # "j_nst_pp_vs_v1" : "JointTraining_NoSelfTraining_PostProcessing_VectorScaling1",
    # "j_nst_pp_vs_v2" : "JointTraining_NoSelfTraining_PostProcessing_VectorScaling0",
    # "j_nst_pp_ms_v1" : "JointTraining_NoSelfTraining_PostProcessing_MatrixScaling1",
    # "j_nst_pp_ms_v2" : "JointTraining_NoSelfTraining_PostProcessing_MatrixScaling0",

    "d_nst_npp_v1" : "DER_NoSelfTraining_NoPostProcessing1",
    "d_nst_npp_v2" : "DER_NoSelfTraining_NoPostProcessing0",
    "d_st_npp_0025_v1" : "DER_SelfTraining_0.025_NoPostProcessing1",
    "d_st_npp_0025_v2" : "DER_SelfTraining_0.025_NoPostProcessing0",
    "d_nst_pp_ts_v1" : "DER_NoSelfTraining_PostProcessing_TemperatureScaling1",
    "d_nst_pp_ts_v2" : "DER_NoSelfTraining_PostProcessing_TemperatureScaling0",
    # "d_nst_pp_vs_v1" : "DER_NoSelfTraining_PostProcessing_VectorScaling1",
    # "d_nst_pp_vs_v2" : "DER_NoSelfTraining_PostProcessing_VectorScaling0",
    # "d_nst_pp_ms_v1" : "DER_NoSelfTraining_PostProcessing_MatrixScaling1",
    # "d_nst_pp_ms_v2" : "DER_NoSelfTraining_PostProcessing_MatrixScaling0",
    "d_nst_pp_ts_md_v1" : "DER_NoSelfTraining_PostProcessing_TemperatureScaling_MixedData1",
    "d_nst_pp_ts_md_v2" : "DER_NoSelfTraining_PostProcessing_TemperatureScaling_MixedData0",
}

In [ ]:
run2label = {
    # "j_nst_npp" : "JointTraining",
    # "j_st_npp_1" : "JointTraining_ST_1",
    # "j_st_npp_01" : "JointTraining_ST_0.1",
    # "j_st_npp_001" : "JointTraining_ST_0.01",
    # "j_st_npp_005" : "JointTraining_ST_0.05",
    # "j_st_npp_0025" : "JointTraining_ST_0.025",
    # "j_st_npp_0075" : "JointTraining_ST_0.075",
    # "j_st_npp_0005" : "JointTraining_ST_0.005",
    # "j_st_npp_00025" : "JointTraining_ST_0.0025",
    # "j_st_npp_00075" : "JointTraining_ST_0.0075",
    # "j_st_npp" : "JointTraining_ST_0.001",
    # "j_nst_pp" : "JointTraining_PostProcessing",

    # "j_nst_npp_v1" : "JointTraining",
    # "j_nst_npp_v2" : "JointTraining",
    # "j_nst_npp_v3" : "JointTraining",
    # "j_st_npp_0025_v1" : "JointTraining_ST_0.025",
    # "j_st_npp_0025_v2" : "JointTraining_ST_0.025",
    # "j_st_npp_0025_v3" : "JointTraining_ST_0.025",

    # "r_nst_npp_v1" : "Replay",
    # "r_nst_npp_v2" : "Replay",
    # "r_nst_npp_v3" : "Replay",

    # "n_nst_npp_v1" : "Naive",
    # "n_nst_npp_v2" : "Naive",
    # "n_nst_npp_v3" : "Naive",

    # # "d_nst_npp_v1" : "DER",
    # # "d_nst_npp_v2" : "DER",
    # # "d_nst_npp_v3" : "DER",

    # "d_nst_npp" : "DER",
    # "d_st_npp_01" : "DER_0.1",
    # "d_st_npp_0075" : "DER_0.075",
    # "d_st_npp_005" : "DER_0.05",
    # "d_st_npp_0025_v1" : "DER_0.025",
    # "d_st_npp_0025_v2" : "DER_0.025",
    # "d_st_npp_0025_v3" : "DER_0.025",
    # "d_st_npp_001" : "DER_0.01",
    # "d_st_npp_00075" : "DER_0.0075",
    # "d_st_npp_0005" : "DER_0.005",
    # "d_st_npp_00025" : "DER_0.0025",

    # "d_nst_npp_hp1" : "DER_001_01_05",
    # "d_nst_npp_hp2" : "DER_001_01_05",
    # "d_nst_npp_hp3" : "DER_003_01_05",
    # "d_nst_npp_hp4" : "DER_003_01_05",
    # "d_nst_npp_hp13" : "DER_001_01_1",
    # "d_nst_npp_hp14" : "DER_001_01_1",
    # "d_nst_npp_hp5" : "DER_001_02_05",
    # "d_nst_npp_hp6" : "DER_001_02_05",
    # "d_nst_npp_hp7" : "DER_001_02_1",
    # "d_nst_npp_hp8" : "DER_001_02_1",
    # "d_nst_npp_hp9" : "DER_001_05_05",
    # "d_nst_npp_hp10" : "DER_001_05_05",
    # "d_nst_npp_hp11" : "DER_001_05_1",
    # "d_nst_npp_hp12" : "DER_001_05_1",

    # "d_nst_npp_m1" : "DER_001_01_05",
    # "d_nst_npp_m2" : "DER_001_01_05_500",
    # "d_nst_npp_m3" : "DER_001_01_05_1000",
    # "d_nst_npp_m4" : "DER_001_01_05_2000",
    # "d_nst_npp_m5" : "DER_001_01_05_4000",
    # "d_nst_npp_m6" : "DER_001_02_05",
    # "d_nst_npp_m7" : "DER_001_02_05_500",
    # "d_nst_npp_m8" : "DER_001_02_05_1000",
    # "d_nst_npp_m9" : "DER_001_02_05_2000",
    # "d_nst_npp_m10" : "DER_001_02_05_4000",

    # "d_nst_npp_v1" : "DER_01_05",
    # "d_nst_npp_v2" : "DER_01_05",
    # "d_nst_npp_v3" : "DER_01_05",
    # "d_nst_npp_v4" : "DER_01_08",
    # "d_nst_npp_v5" : "DER_01_08",
    # "d_nst_npp_v6" : "DER_01_08",
    # "d_nst_npp_v7" : "DER_02_05",
    # "d_nst_npp_v8" : "DER_02_05",
    # "d_nst_npp_v9" : "DER_02_05",
    # "d_nst_npp_v10" : "DER_02_08",
    # "d_nst_npp_v11" : "DER_02_08",
    # "d_nst_npp_v12" : "DER_02_08",
    # "d_nst_npp_v13" : "DER_03_05",
    # "d_nst_npp_v14" : "DER_03_05",
    # "d_nst_npp_v15" : "DER_03_05",
    # "d_nst_npp_v16" : "DER_03_08",
    # "d_nst_npp_v17" : "DER_03_08",
    # "d_nst_npp_v18" : "DER_03_08",
    # "d_nst_npp_v19" : "DER_05_05",
    # "d_nst_npp_v20" : "DER_05_05",
    # "d_nst_npp_v21" : "DER_05_05",
    # "d_nst_npp_v22" : "DER_05_08",
    # "d_nst_npp_v23" : "DER_05_08",
    # "d_nst_npp_v24" : "DER_05_08",

    "j_nst_npp_v1" : "J",
    "j_nst_npp_v2" : "J",
    "j_st_npp_0025_v1" : "J + HR",
    "j_st_npp_0025_v2" : "J + HR",
    "j_nst_pp_ts_v1" : "J + TS",
    "j_nst_pp_ts_v2" : "J + TS",
    "j_nst_pp_vs_v1" : "J + VS",
    "j_nst_pp_vs_v2" : "J + VS",
    "j_nst_pp_ms_v1" : "J + MS",
    "j_nst_pp_ms_v2" : "J + MS",

    "d_nst_npp_v1" : "D",
    "d_nst_npp_v2" : "D",
    "d_st_npp_0025_v1" : "D + HR",
    "d_st_npp_0025_v2" : "D + HR",
    "d_nst_pp_ts_v1" : "D + TS",
    "d_nst_pp_ts_v2" : "D + TS",
    # "d_nst_pp_vs_v1" : "DER_NoSelfTraining_PostProcessing_VectorScaling1",
    # "d_nst_pp_vs_v2" : "DER_NoSelfTraining_PostProcessing_VectorScaling0",
    # "d_nst_pp_ms_v1" : "DER_NoSelfTraining_PostProcessing_MatrixScaling1",
    # "d_nst_pp_ms_v2" : "DER_NoSelfTraining_PostProcessing_MatrixScaling0",
    "d_nst_pp_ts_md_v1" : "D + TS + MD",
    "d_nst_pp_ts_md_v2" : "D + TS + MD",
}

In [ ]:
running_accuracy = []
running_ece = []
final_accuracy = []
final_ece = []
bins = None
ece_hist_vals = []

for k, name in run2name.items():
    print(f">> {name} <<")
    with open(f"{DATA_PATH}/{DATASET}_{MODEL_NAME}_{name}_dict", "rb") as file:
        data = pickle.load(file)

        # print("\n---- ACCURACY ----")

        metric_str = "Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp"
        m = []
        for i in range(len(data)):
            cur_exp_dict = data[i]
            cur_exp_acc = 0
            # compute the average over the experiences trained so far (i)
            for j in range(i+1):
                # print(i, j, metric_str + f"{j:03d}", cur_exp_dict[metric_str + f"{j:03d}"])
                cur_exp_acc += cur_exp_dict[metric_str + f"{j:03d}"]
            m.append(cur_exp_acc/(i+1))
        
        # duplicate for JointTraining
        if len(m) < NUM_EXPERIENCES:
            m = m*NUM_EXPERIENCES
        running_accuracy.append((k, m))
        final_accuracy.append((k, running_accuracy[-1][-1][-1]))
        # print(k, m, running_accuracy[-1][-1][-1])

        # print("\n---- ECE ----")

        metric_str = "ECE_Exp/eval_phase/test_stream/Task000/Exp"
        m = []
        for i in range(len(data)):
            cur_exp_dict = data[i]
            cur_exp_ece = 0
            # compute the average over the experiences trained so far (i)
            for j in range(i+1):
                # print(i, j, metric_str + f"{j:03d}", cur_exp_dict[metric_str + f"{j:03d}"])
                cur_exp_ece += cur_exp_dict[metric_str + f"{j:03d}"]
            # m.append(cur_exp_ece/(i+1))
            m.append((cur_exp_ece/(i+1))*100)
        
        # duplicate for JointTraining
        if len(m) < NUM_EXPERIENCES:
            m = m*NUM_EXPERIENCES
        running_ece.append((k, m))
        final_ece.append((k, running_ece[-1][-1][-1]))
        # print(k, m, running_ece[-1][-1][-1])

        # print("\n---- ECE HISTOGRAMS ----")

        metric_str = "ExpECEHistogram/eval_phase/test_stream/Exp"
        m = []
        i = -1 # after last experience
        cur_exp_dict = data[i]
        for j in range(NUM_EXPERIENCES):
            # print(i, j, metric_str + f"{j:03d}", cur_exp_dict[metric_str + f"{j:03d}"])
            fig = cur_exp_dict[metric_str + f"{j:03d}"]
            axes_list = fig.get_axes()
            for ax in axes_list:
                for line in ax.get_lines()[-1:]:
                    x_data = line.get_xdata()
                    y_data = line.get_ydata()
                    # print({'x': x_data, 'y': y_data})
                    if bins is None:
                        bins = x_data
                    m.append(y_data)
        # print(bins, m)
        bin_vals = []
        for i in range(len(bins)):
            x = []
            for j in range(len(m)):
                # print(j, i, m[j][i])
                x.append(m[j][i])
            mean = np.mean(x)
            std = np.std(x)
            bin_vals.append((mean, std))
        # print(k, bin_vals)
        ece_hist_vals.append((k, bin_vals))


In [ ]:
# PLOT 1: Average accuracy on all experiences after training on exp j

plt.figure(figsize=(7, 6))
x_axis = list(range(1, NUM_EXPERIENCES+1))
for i, (name, vals) in enumerate(running_accuracy):
    plt.plot(x_axis, vals, label=run2label[name], color=valid_colors[i])
plt.title('Average Experience Accuracy')
plt.xlabel('#Trained Experience')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
plt.xlim(1, NUM_EXPERIENCES)
plt.xticks(x_axis, x_axis)
plt.legend(loc='upper right', fontsize='small', ncol=2)
plt.savefig(f'./imgs/{DATASET}_{NUM_EXPERIENCES}_{str.lower("Average_Experience_Accuracy")}.png', dpi=400)
plt.show()

In [ ]:
# PLOT 2: Average ece on all experiences after training on exp j

plt.figure(figsize=(7, 6))
x_axis = list(range(1, NUM_EXPERIENCES+1))
for i, (name, vals) in enumerate(running_ece):
    plt.plot(x_axis, vals, label=run2label[name], color=valid_colors[i])
plt.title('Average Experience ECE')
plt.xlabel('#Trained Experience')
plt.ylabel('ECE')
# plt.ylim(0, 1)
plt.xlim(1, NUM_EXPERIENCES)
plt.xticks(x_axis, x_axis)
plt.legend(loc='upper right', fontsize='small', ncol=2)
plt.savefig(f'./imgs/{DATASET}_{NUM_EXPERIENCES}_{str.lower("Average_Experience_ECE")}.png', dpi=400)
plt.show()

In [ ]:
# TABLE : average accuracy/ece on all experiences at the end of training

table_data = []
for (n, acc), (_, ece) in zip(final_accuracy, final_ece):
    table_data.append((run2label[n], round(acc*100, 2), round(ece, 4)))

dt = pd.DataFrame(table_data, columns=["RunName", "Accuracy", "ECE"])
print(dt)

In [ ]:
# Calculate mean and standard deviation for each run
mean_accuracy_per_run = dt.groupby('RunName')['Accuracy'].mean()
std_accuracy_per_run = dt.groupby('RunName')['Accuracy'].std()

mean_ece_per_run = dt.groupby('RunName')['ECE'].mean()
std_ece_per_run = dt.groupby('RunName')['ECE'].std()

# Print the results
print("Mean Accuracy per Run:\n", mean_accuracy_per_run)
print("\nStandard Deviation of Accuracy per Run:\n", std_accuracy_per_run)

print("\nMean ECE per Run:\n", mean_ece_per_run)
print("\nStandard Deviation of ECE per Run:\n", std_ece_per_run)

In [ ]:
print(dt.to_latex(index=False, float_format="{:.2f}".format))

In [ ]:
# HISTOGRAM : avg/std across all experiences at the end of training

fig, axs = plt.subplots(3, 4, figsize=(12, 8))
axs = axs.flatten()
for i, (name, vals) in enumerate(ece_hist_vals):
    m = [e[0] for e in vals]
    s = [e[1] for e in vals]
    l = [max(e[0] - e[1], 0) for e in vals] # cap lower-bound at zero
    u = [e[0] + e[1] for e in vals]
    axs[i].plot([0, 1], [0, 1], '--', label='ideal')
    # axs[i].plot(bins, m, color=valid_colors[i])
    # axs[i].fill_between(bins, l, u, alpha=0.3, linestyle='--', color=valid_colors[i])
    axs[i].errorbar(bins, m, yerr=s, marker="o", linestyle="--", capsize=3, capthick=1, color=valid_colors[i])
    axs[i].set_ylim(-0.05, 1)
    axs[i].set_xlim(0, 1)
    axs[i].set_ylabel("Accuracy")
    axs[i].set_xlabel("Confidence")
    # axs[i].legend(loc='upper left', fontsize='small')
    axs[i].set_title(run2label[name])
plt.tight_layout()
plt.savefig(f'./imgs/{DATASET}_{NUM_EXPERIENCES}_avg_std_calibration.png', dpi=400)
plt.show()

In [ ]:
plt.figure(figsize=(10, 9))
plt.plot([0, 1], [0, 1], '--', label='ideal')
for i, (name, vals) in enumerate(ece_hist_vals):
    m = [e[0] for e in vals]
    s = [e[1] for e in vals]
    plt.errorbar(bins, m, yerr=s, marker="o", linestyle="--", capsize=3, capthick=1, color=valid_colors[i], label=run2label[name])
plt.legend(loc='best', fontsize='small')
plt.show()